In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from scipy import stats
linkedin = pd.read_csv("linkedin_company.csv",low_memory=False)
linkedin=linkedin.drop(['link','date_added','date_updated','description','website','entity_id','cusip','isin'],axis=1)
linkedin.head()

,dataset_id,as_of_date,company_name,followers_count,employees_on_platform,industry
0,58329,2015-09-14,Goldman Sachs,552254,38124,Investment Banking
1,58329,2015-09-15,Goldman Sachs,552862,38141,Investment Banking
2,58363,2015-09-16,United Technologies,59157,14982,Aviation & Aerospace
3,58366,2015-09-16,Novo Nordisk,336175,26448,Pharmaceuticals
4,58371,2015-09-16,"Lowe's Companies, Inc.",134255,62574,Retail


In [5]:
Now=linkedin[linkedin['as_of_date']>='2018-01-01'].drop_duplicates(subset='dataset_id',keep='last')
Past=linkedin[linkedin['as_of_date']<'2016-01-01'].drop_duplicates(subset='dataset_id',keep='first')
Past=Past.sort_values('followers_count',ascending=False).reset_index(drop=True)
Past.loc[:,'rank_2015']=Past.index
Top10_Now=Now.sort_values('followers_count',ascending=False).head(10)
Top10=pd.merge(Top10_Now,Past,how='left',on='dataset_id')
Top10.followers_count_y.fillna(0,inplace=True)
Top10.loc[:,'rank_increase']=Top10.rank_2015-Top10.index
Top10.rank_increase.fillna('NA',inplace=True)
trace1=go.Bar(
            y=[Top10.company_name_x,Top10.rank_increase],
            x=Top10.followers_count_x,
            name='2018',
            orientation='h'
            )

trace2=go.Bar(
            y=[Top10.company_name_x,Top10.rank_increase],
            x=Top10.followers_count_y,
            name='2015',
            orientation='h'
            )
layout=go.Layout(
                barmode='group',
                title='2018 TOP 10 followed company at Linkedin',
                yaxis=dict(autorange='reversed'),
                xaxis=dict(title='Followers Count'),
                )

iplot(go.Figure(data=[trace1,trace2],layout=layout))

,dataset_id,as_of_date_x,company_name_x,followers_count_x,employees_on_platform_x,industry_x,as_of_date_y,company_name_y,followers_count_y,employees_on_platform_y,industry_y,rank_2015,rank_increase
0,58448,2018-07-17,Google,7833967,140679,Internet,2015-09-16,Google,3615989.0,67682.0,Internet,0.0,0
1,58362,2018-07-17,Microsoft,5563173,152462,Computer Software,2015-09-16,Microsoft,2539829.0,123785.0,Computer Software,3.0,2
2,58443,2018-07-17,Amazon,5188572,261440,Internet,2015-09-16,Amazon,1311936.0,84671.0,Internet,10.0,8
3,58406,2018-07-17,Apple,4743555,186050,Consumer Electronics,2015-09-16,Apple,2587280.0,103924.0,Consumer Electronics,1.0,-2
4,58434,2018-07-17,IBM,4525919,577952,Information Technology and Services,2015-09-16,IBM,2542182.0,414139.0,Information Technology and Services,2.0,-2
5,868702,2018-07-17,The Wall Street Journal,4301136,6961,Newspapers,NaN,NaN,0.0,NaN,NaN,NaN,NA
6,58598,2018-07-17,LinkedIn,3486823,16360,Internet,2015-09-17,LinkedIn,1422976.0,10311.0,Internet,8.0,2
7,58394,2018-07-17,Accenture,3409892,380166,Information Technology and Services,2015-09-16,Accenture,2063480.0,258009.0,Information Technology and Services,5.0,-2
8,58376,2018-07-17,Unilever,3335998,111102,Consumer Goods,2015-09-16,Unilever,1719887.0,76272.0,Consumer Goods,6.0,-2
9,890665,2018-07-17,The New York Times,3089557,5962,Newspapers,NaN,NaN,0.0,NaN,NaN,NaN,NA


In [16]:
y=Now.groupby('industry')['followers_count'].median().sort_values()
x=Now.groupby('industry')['employees_on_platform'].median().sort_values()
data=[go.Scatter(x=x,y=y,mode='markers')]
layout=go.Layout(title='2018 Industry Median Followers Count vs. Employees on Platform Count',
                xaxis=dict(title='Employees_on_Platform Count',range=[0,5000]),
                yaxis=dict(title='Followers Count',range=[0,80000]))
iplot(go.Figure(data=data,layout=layout))